In [ ]:
using Zygote
using JuMP
using Ipopt
using LinearAlgebra
using MathOptInterface
const MOI = MathOptInterface
using SparseArrays
using BenchmarkTools
using Plots

# Feasibility restoration phase function:

In [ ]:
function restoration_phase(xk, ρ)
    println("Solving restoration problem")
    
    is_s_eq_0 = false
    
    # Define nonlinear model in JuMP
    nlrp = Model()
    set_silent(nlrp)
    @variable(nlrp, x[i=1:2])
    @NLobjective(nlrp, Min, (x[1] - 2)^2 + (x[2] - 2)^2)
    @NLconstraint(nlrp, c, x[1]^2 + x[2]^2 - 1 <= 0)
    
    # Initialise MOI evaluator
    pp = NLPEvaluator(nlrp)
    MOI.initialize(pp, [:Jac,:Grad,:Hess])

    # Iterate until convergence
    while !is_s_eq_0
        
        #Constraint eval
        ggg=zeros(1)
        MOI.eval_constraint(pp,ggg,xk)

        #Jacobian eval
        JStru=MOI.jacobian_structure(pp)
        Ja=zeros(length(JStru))
        MOI.eval_constraint_jacobian(pp,Ja,xk)

        jr = Vector{Int64}(undef, length(Ja)); jc = Vector{Int64}(undef, length(Ja))
        for i in 1:length(JStru)
            jr[i] = JStru[i][1] |> Int
            jc[i] = JStru[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, Ja) 
        @assert size(Jacobeval, 1) == 1
                                        
        Jacobeval = vcat(Jacobeval...)
        
        # Linearised restoration phase model
        restor_model = Model(Ipopt.Optimizer)
        set_silent(restor_model)
        @variable(restor_model, s >= 0)
        @variable(restor_model, d[i=1:2])

        @objective(restor_model, Min, s) 
        @constraint(restor_model, c1, ggg[1] + Jacobeval'*d - s <= 0)
        @constraint(restor_model, c2, d[1] <= ρ)
        @constraint(restor_model, c3, d[2] <= ρ) 
        @constraint(restor_model, c4, d[1] >= -ρ)
        @constraint(restor_model, c5, d[2] >= -ρ)
        optimize!(restor_model)

        proposed_xk = xk .+ value.(d)
        
        # merit function
        function m(x::Vector{<:Real}) 
            ce = zeros(1)
            MOI.eval_constraint(pp,ce,x)
            merit = norm(max(ce[1], 0))
            return merit
        end

        if abs(value(s)) < 1e-6
        elseif m(proposed_xk) < m(xk) 
            println("restoration merit condition: success")
            xk = proposed_xk
            ρ += 1*ρ
        else
            println("restoration merit condition: failed")
            ρ -= 0.5*ρ
        end
        
        println("slack = $(value(s))")
        is_s_eq_0 = abs(value(s)) < 1e-6
    end
    
    return xk, ρ 
end

# SLP with a merit function:

In [ ]:
function SLP_MF(xk::Vector{<:Real}, ρ, γ)
    # ρ and γ are parameters
    @assert γ>=0
    
    has_x_converged = false
    ctr = 0
    restor_ctr = 0

    # Nonlinear model in JuMP
    nlmod = Model()
    set_silent(nlmod)
    @variable(nlmod, x[i=1:2])
    @NLobjective(nlmod, Min, (x[1] - 2)^2 + (x[2] - 2)^2)
    @NLconstraint(nlmod, c, x[1]^2 + x[2]^2 - 1 <= 0)
    
    # Initialise MOI evaluator
    p = NLPEvaluator(nlmod)
    MOI.initialize(p, [:Jac,:Grad,:Hess])
    

    while has_x_converged != [1, 1]
        ctr += 1
        
        #Objective eval
        objEv=MOI.eval_objective(p, xk)

        #Objective gradient
        df=zeros(length(xk))
        MOI.eval_objective_gradient(p,df,xk)

        #Constraint eval
        gg=zeros(1)
        MOI.eval_constraint(p,gg,xk)

        #Jacobian eval
        JStr=MOI.jacobian_structure(p)
        J=zeros(length(JStr))
        MOI.eval_constraint_jacobian(p,J,xk)

        jr = Vector{Int64}(undef, length(J)); jc = Vector{Int64}(undef, length(J))
        for i in 1:length(JStr)
            jr[i] = JStr[i][1] |> Int
            jc[i] = JStr[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, J) 
        @assert size(Jacobeval, 1) == 1                              
        Jacobeval = vcat(Jacobeval...)

        # Define linearised model in JuMP
        model = Model(Ipopt.Optimizer)
        set_silent(model)
        @variable(model, d[i=1:2])
        @objective(model, Min, objEv + df'*d)
        @constraint(model, c1,  gg[1] + Jacobeval'*d <= 0)
        @constraint(model, c2, d[1] <= ρ)
        @constraint(model, c3, d[2] <= ρ) 
        @constraint(model, c4, d[1] >= -ρ)
        @constraint(model, c5, d[2] >= -ρ)
        
        println("Iteration $ctr...")

        optimize!(model)
        @show termination_status(model)
        
        # update step
        proposed_xk = xk .+ value.(d)
        
        # merit function
        function m(x::Vector{<:Real}) 
            newobjev = MOI.eval_objective(p, x)
            ce = zeros(1)
            MOI.eval_constraint(p,ce,x)
            merit = newobjev + γ * norm(max(ce[1], 0))
            return merit
        end
        
        println("m(xk) = $(m(xk))")
        println("m(prop_xk) = $(m(proposed_xk))")
        
        if m(proposed_xk) < m(xk)
            println("merit condition: success")
            xk = proposed_xk
            ρ += 1*ρ
        else
            println("merit condition: failed")
            ρ -= 0.5*ρ
        end
        
        println("d=$(value.(d))")
        println("x$(ctr)=$(xk)")
        
        # restoration phase:
        if termination_status(model) == MathOptInterface.LOCALLY_INFEASIBLE
            xk, ρ = restoration_phase(xk, ρ)
            restor_ctr += 1
        end
        
        #convergence test
        tol = 1e-6
        has_x_converged = abs.(value.(d)) .< tol
        
    end
    println("no. restoration phases: $(restor_ctr)")
end

In [ ]:
SLP_MF([2., 2.], 4, 1)

# SLP with a filter function:

In [ ]:
function SLP_F(xk::Vector{<:Real}, ρ)
    # ρ is the trust region parameter

    has_x_converged = false
    ctr = 0
    restor_ctr = 0
    
    filt = []
    
    # Define nonlinear model in JuMP
    nlmod = Model()
    set_silent(nlmod)
    @variable(nlmod, x[i=1:2])
    @NLobjective(nlmod, Min, (x[1] - 2)^2 + (x[2] - 2)^2)
    @NLconstraint(nlmod, c, x[1]^2 + x[2]^2 - 1 <= 0)
    
    # Initialise MOI evaluator
    p = NLPEvaluator(nlmod)
    MOI.initialize(p, [:Jac,:Grad,:Hess])
    
    while has_x_converged != [1, 1]
        ctr += 1
        
        #Objective eval
        objEv=MOI.eval_objective(p, xk)

        #Objective gradient
        df=zeros(length(xk))
        MOI.eval_objective_gradient(p,df,xk)

        #Constraint eval
        gg=zeros(1)
        MOI.eval_constraint(p,gg,xk)

        #Jacobian eval
        JStr=MOI.jacobian_structure(p)
        J=zeros(length(JStr))
        MOI.eval_constraint_jacobian(p,J,xk)

        jr = Vector{Int64}(undef, length(J)); jc = Vector{Int64}(undef, length(J))
        for i in 1:length(JStr)
            jr[i] = JStr[i][1] |> Int
            jc[i] = JStr[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, J) 
        @assert size(Jacobeval, 1) == 1                              
        Jacobeval = vcat(Jacobeval...)
        
        # Define linearised model in JuMP
        model = Model(Ipopt.Optimizer)
        set_silent(model)
        @variable(model, d[i=1:2])

        println("Iteration $ctr...")

        @objective(model, Min, objEv + df'*d) 
        @constraint(model, c1, gg[1] + Jacobeval'*d <= 0)
        @constraint(model, c2, d[1] <= ρ)
        @constraint(model, c3, d[2] <= ρ)
        @constraint(model, c4, d[1] >= -ρ)
        @constraint(model, c5, d[2] >= -ρ)

        optimize!(model)
        
        #functions for making the filter
        function fk(x::Vector{<:Real}) 
            newobjev = MOI.eval_objective(p, x)
            return newobjev
        end
        function hk(x::Vector{<:Real}) 
            ce = zeros(1)
            MOI.eval_constraint(p,ce,x)
            h = norm(max(ce[1], 0))
            return h
        end
        
        proposed_xk = xk .+ value.(d)
        better_obj = all(v -> fk(proposed_xk) < v[1], filt)
        better_constr = all(v -> hk(proposed_xk) < v[2], filt)
        
        if better_obj || better_constr
            println("filter: accept")
            xk = proposed_xk
            ρ += 1*ρ
            filt = push!(filt, (fk(xk), hk(xk)))
        else
            println("filter: reject")
            ρ -= 0.5*ρ
            filt = push!(filt, (fk(proposed_xk), hk(proposed_xk)))
        end
        
        
        println("f(propxk) = $(fk(proposed_xk))")
        println("d=$(value.(d))")
        println("x$(ctr)=$xk")  
        
        #restoration phase:
        if termination_status(model) == MathOptInterface.LOCALLY_INFEASIBLE
            xk, ρ = restoration_phase(xk, ρ)
            restor_ctr += 1
        end
    
        #convergence test
        tol = 1e-6
        has_x_converged = abs.(value.(d)) .< tol
    end
    println("no. restoration phases: $(restor_ctr)")
end


In [ ]:
SLP_F([100., -100.], 4)

# SQP with a merit function:

In [ ]:
function SQP_MF(xk::Vector{<:Real}, λk::Vector{Float64}, ρ, γ)
    # λ, μ are initial lagrangian multipliers
    # ρ and γ are parameters

    has_x_converged = false
    ctr = 0
    restor_ctr = 0
    
    # Nonlinear model
    mhl = Model(Ipopt.Optimizer)
    @variable(mhl, x[i=1:2])
    @NLobjective(mhl, Min, (x[1] - 2)^2 + (x[2] - 2)^2 )
    @NLconstraint(mhl, c, x[1]^2 + x[2]^2 - 1 <= 0)
    p = NLPEvaluator(mhl)
    MOI.initialize(p, [:Jac,:Grad,:Hess])
    
    while has_x_converged != [1, 1]
        ctr += 1
        
        #Objective eval
        objEv=MOI.eval_objective(p, xk)

        #Objective gradient
        df=zeros(length(xk))
        MOI.eval_objective_gradient(p,df,xk)

        #Constraint eval
        gg=zeros(1)
        MOI.eval_constraint(p,gg,xk)

        #Jacobian eval
        JStr=MOI.jacobian_structure(p)
        J=zeros(length(JStr))
        MOI.eval_constraint_jacobian(p,J,xk)

        jr = Vector{Int64}(undef, length(J)); jc = Vector{Int64}(undef, length(J))
        for i in 1:length(JStr)
            jr[i] = JStr[i][1] |> Int
            jc[i] = JStr[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, J) 
        @assert size(Jacobeval, 1) == 1                              
        Jacobeval = vcat(Jacobeval...)
        
        #Hessian-of-the-Lagrangian eval
        HStr=MOI.hessian_lagrangian_structure(p)
        H=zeros(length(HStr))
        MOI.eval_hessian_lagrangian(p,H,xk,1.0,repeat(λk,length(H))) # instead of repeat have lagrange multipliers
        
        hr = Vector{Int64}(undef, length(H)); hc = Vector{Int64}(undef, length(H))
        for i in 1:length(HStr)
            hr[i] = HStr[i][1] |> Int
            hc[i] = HStr[i][2] |> Int
        end
        
        HessLag = sparse(hr, hc, H)
        
        hessianobj(d) = 0.5*d'*(HessLag*d)
        
        # Linearised model
        model = Model(Ipopt.Optimizer)
        set_silent(model)
        @variable(model, d[i=1:2])
        @objective(model, Min, objEv + df'*d + hessianobj(d)) 
        @constraint(model, c1, gg[1] + Jacobeval'*d <= 0)
        @constraint(model, c2, d[1] <= ρ)
        @constraint(model, c3, d[2] <= ρ)
        @constraint(model, c4, d[1] >= -ρ)
        @constraint(model, c5, d[2] >= -ρ)
        
        println("Iteration $ctr...")

        optimize!(model)
        @show termination_status(model)
        
        # update λ
        λk[1] = shadow_price(c1)
        println("lambda$(ctr) = $(λk[1])")

        # merit function
        function m(x::Vector{<:Real}) 
            newobjev = MOI.eval_objective(p, x)
            ce = zeros(1)
            MOI.eval_constraint(p,ce,x)
            merit = newobjev + γ * norm(max(ce[1], 0))
            return merit
        end
        
        # updating ρ
        proposed_xk = xk .+ value.(d)
        
        println("m(prop_xk) = $(m(proposed_xk))")
        println("m(xk) = $(m(xk))")
        
        if m(proposed_xk) < m(xk)
            println("merit condition: success")
            xk = proposed_xk
            ρ += 1*ρ
        else
            println("merit condition: failed")
            ρ -= 0.5*ρ
        end
        
        println("d=$(value.(d))")
        println("x$ctr=$xk")
        
        #restoration phase:
        if termination_status(model) == MathOptInterface.LOCALLY_INFEASIBLE
            xk, ρ = restoration_phase(xk, ρ)
            restor_ctr += 1
        end
        
        #convergence test
        tol = 1e-6
        has_x_converged = abs.(value.(d)) .< tol
    end
    println("no. restoration phases: $(restor_ctr)")
end

In [ ]:
SQP_MF([2., 2.], [1.], 4, 2)

# SQP with a filter function:

In [ ]:
function SQP_F(xk::Vector{<:Real}, λk::Vector{Float64}, ρ)
    # λ is initial lagrangian multiplier
    # ρ is the trust region parameter

    has_x_converged = false
    ctr = 0
    restor_ctr = 0
    
    filt = []
    
    # Nonlinear model
    mhl = Model(Ipopt.Optimizer)
    @variable(mhl, x[i=1:2])
    @NLobjective(mhl, Min, (x[1] - 2)^2 + (x[2] - 2)^2 )
    @NLconstraint(mhl, c1, x[1]^2 + x[2]^2 - 1 <= 0)
    p = NLPEvaluator(mhl)
    MOI.initialize(p, [:Jac,:Grad,:Hess])
    
    while has_x_converged != [1, 1]
        ctr += 1
        
        #Objective eval
        objEv=MOI.eval_objective(p, xk)

        #Objective gradient
        df=zeros(length(xk))
        MOI.eval_objective_gradient(p,df,xk)

        #Constraint eval
        gg=zeros(1)
        MOI.eval_constraint(p,gg,xk)

        #Jacobian eval
        JStr=MOI.jacobian_structure(p)
        J=zeros(length(JStr))
        MOI.eval_constraint_jacobian(p,J,xk)

        jr = Vector{Int64}(undef, length(J)); jc = Vector{Int64}(undef, length(J))
        for i in 1:length(JStr)
            jr[i] = JStr[i][1] |> Int
            jc[i] = JStr[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, J) 
        @assert size(Jacobeval, 1) == 1                              
        Jacobeval = vcat(Jacobeval...)
        
        #Hessian-of-the-Lagrangian eval
        HStr=MOI.hessian_lagrangian_structure(p)
        H=zeros(length(HStr))
        MOI.eval_hessian_lagrangian(p,H,xk,1.0,repeat(λk,length(H))) 
        
        hr = Vector{Int64}(undef, length(H)); hc = Vector{Int64}(undef, length(H))
        for i in 1:length(HStr)
            hr[i] = HStr[i][1] |> Int
            hc[i] = HStr[i][2] |> Int
        end
        HessLag = sparse(hr, hc, H)
        
        hessianobj(d) = 0.5*d'*(HessLag*d)
        
        model = Model(Ipopt.Optimizer)
        set_silent(model)
        @variable(model, d[i=1:2])
        @objective(model, Min, objEv + df'*d + hessianobj(d)) 
        @constraint(model, c1, gg[1] + Jacobeval'*d <= 0)
        @constraint(model, c2, d[1] <= ρ)
        @constraint(model, c3, d[2] <= ρ)
        @constraint(model, c4, d[1] >= -ρ)
        @constraint(model, c5, d[2] >= -ρ)

                
        println("Iteration $ctr...")
     
        optimize!(model)
        
        # Update λ
        λk[1] = shadow_price(c1)

        # Functions for making the filter
        function fk(x::Vector{<:Real}) 
            newobjev = MOI.eval_objective(p, x)
            return newobjev
        end
        function hk(x::Vector{<:Real}) 
            ce = zeros(1)
            MOI.eval_constraint(p,ce,x)
            h = norm(max(ce[1], 0))
            return h
        end
        
        # Update step
        proposed_xk = xk .+ value.(d)
        better_obj = all(v -> fk(proposed_xk) < v[1], filt)
        better_constr = all(v -> hk(proposed_xk) < v[2], filt)
        
        if better_obj || better_constr
            println("filter: accept")
            xk = proposed_xk
            ρ += 1*ρ
            filter = push!(filt, (fk(xk), hk(xk)))
        else
            println("filter: reject")
            ρ -= 0.5*ρ
            filter = push!(filt, (fk(proposed_xk), hk(proposed_xk)))
        end
        
        println("f(propxk) = $(fk(proposed_xk))")
        println("d=$(value.(d))")
        println("x$ctr=$xk")
        
        #restoration phase:
        if termination_status(model) == MathOptInterface.LOCALLY_INFEASIBLE
            xk, ρ = restoration_phase(xk, ρ)
            restor_ctr += 1
        end
        
        #convergence test
        tol = 1e-6
        has_x_converged = abs.(value.(d)) .< tol
    end
    println("no. restoration phases: $(restor_ctr)")
end

In [ ]:
SQP_F([100., -100.], [1.], 4)